In [1]:
import numpy as np, pandas as pd
import sympy as sym
from sympy import symbols, Matrix, solve, simplify
import math

In [2]:
#### initialize unknown variable symbols (30 in total)

# susceptible to diarrhea prevalence variables
p_S1, p_S2, p_S3, p_S4 = symbols('p_S1 p_S2 p_S3 p_S4')

# infected with diarrhea prevalence variables
p_D1, p_D2, p_D3, p_D4 = symbols('p_D1 p_D2 p_D3 p_D4')

In [4]:
unknowns = [p_S1, p_S2, p_S3, 
           p_D1, p_D2, p_D3, p_S4, p_D4
]
len(unknowns)

8

In [5]:
### initialize known variable symbols

# diarrhea cause model variables
prevalence_c302 = symbols('prevalence_c302') 

# wasting risk exposure values
p_4, p_3, p_2, p_1 = symbols('p_4 p_3 p_2 p_1')

# wasting exposure prevalence ratios by diarrheal state
PR_1, PR_2, PR_3, PR_4 = symbols('PR_1 PR_2 PR_3 PR_4')

In [6]:
#[{unknown:coefficient, unknown:coefficient}, right hand side]

eq1 = [{p_D1:1/prevalence_c302, p_S1:-PR_1/(1-prevalence_c302)}, 0]
eq2 = [{p_D2:1/prevalence_c302, p_S2:-PR_2/(1-prevalence_c302)}, 0]
eq3 = [{p_D3:1/prevalence_c302, p_S3:-PR_3/(1-prevalence_c302)}, 0]
eq4 = [{p_D4:1/prevalence_c302, p_S4:-PR_4/(1-prevalence_c302)}, 0]
eq5 = [{p_D1:1, p_D2:1, p_D3:1, p_D4:1}, prevalence_c302]
eq6 = [{p_S1:1, p_D1:1}, p_1]
eq7 = [{p_S2:1, p_D2:1}, p_2]
eq8 = [{p_S3:1, p_D3:1}, p_3]

In [7]:
eqs = [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8]
len(eqs)

8

In [8]:
def build_matrix(eqns, unknowns):
    """
    INPUT
    ----
    eqns: a list of equations. These are of the form:
          [{x_i:a, x_j:b}, y] for the equation:
          ax_i + bx_j = y
    unknowns: a list of sympy unknowns
    ----
    OUTPUT
    ----
    A:  a matrix containing the coefficients of left-hand side of all eq in eqns.
        - nrows = number of equations
        - rcols = number of unknowns
    b: an nx1 matrix containing the right-hand side of all the eqns
    x: a sympy matrix of the unknowns
    
    Note that Ax = b
    """
    n_eqns = len(eqns)
    n_unknowns = len(unknowns)

    # frame for matrix/LHS equation coefficients.
    # nrows = n_eqns, ncols = n_unknowns
    A = pd.DataFrame(
        index = range(n_eqns),
        columns = unknowns,
        data = np.zeros([n_eqns,n_unknowns])
    )

    # frame for RHS of equations
    b = pd.DataFrame(index = range(n_eqns), columns = ['val'])
    
    # define helper fn
    def add_eq(terms, y, i):
        """
        INPUT
        ----
        To input i^(th) equation y = ax_i + bx_j, add_eq wants:
            - terms = {x_i:a, x_j:b}
            - y = y
            - i
        ----
        FUNCTION
        ----
        - Adds coefficients to matrix A.
        - Adds y to row i of vector b
        ----
        EXAMPLE
        ----
        For y = ax_i + bx_j:
        - adds "a" to column "x_i" row i of A
        - adds "b" to column "x_j" row i of A
        - adds "y" to row i of b
        """
        for x in terms.keys():
            A[x][i] = terms[x]
        b.iloc[i] = y

    # populate LHS/RHS
    i = 0
    for eq in eqns:
        add_eq(terms=eq[0], y=eq[1], i=i)
        i += 1

    # convert to sympy matrices
    A = sym.Matrix(A)
    b = sym.Matrix(b)
    x = sym.Matrix(unknowns) #vars to solve for

    return A, x, b

In [9]:
A1, x1, b1 = build_matrix(eqs, unknowns)

/ihme/code/alibow/miniconda/envs/vivarium_ciff_sam/lib/python3.6/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
A1

Matrix([
[-PR_1/(1 - prevalence_c302),                         0.0,                         0.0, 1/prevalence_c302,               0.0,               0.0,                         0.0,               0.0],
[                        0.0, -PR_2/(1 - prevalence_c302),                         0.0,               0.0, 1/prevalence_c302,               0.0,                         0.0,               0.0],
[                        0.0,                         0.0, -PR_3/(1 - prevalence_c302),               0.0,               0.0, 1/prevalence_c302,                         0.0,               0.0],
[                        0.0,                         0.0,                         0.0,               0.0,               0.0,               0.0, -PR_4/(1 - prevalence_c302), 1/prevalence_c302],
[                        0.0,                         0.0,                         0.0,                 1,                 1,                 1,                         0.0,                 1],
[                    

In [11]:
A1.shape

(8, 8)

In [12]:
x1

Matrix([
[p_S1],
[p_S2],
[p_S3],
[p_D1],
[p_D2],
[p_D3],
[p_S4],
[p_D4]])

In [13]:
x1.shape

(8, 1)

In [14]:
b1

Matrix([
[              0],
[              0],
[              0],
[              0],
[prevalence_c302],
[            p_1],
[            p_2],
[            p_3]])

In [15]:
b1.shape

(8, 1)

In [17]:
result = sym.solve(A1 * x1 - b1, x1)#, dict=True, set=True)
result
# NOTE: the p_S4 and p_D4 equations are complicated if they do not use
# any of the other unknown variables, but can be simplified by using 
# those values

{p_S1: (-p_1*prevalence_c302 + p_1)/(PR_1*prevalence_c302 - prevalence_c302 + 1),
 p_S2: (-p_2*prevalence_c302 + p_2)/(PR_2*prevalence_c302 - prevalence_c302 + 1),
 p_S3: (-p_3*prevalence_c302 + p_3)/(PR_3*prevalence_c302 - prevalence_c302 + 1),
 p_D1: PR_1*p_1*prevalence_c302/(PR_1*prevalence_c302 - prevalence_c302 + 1),
 p_D2: PR_2*p_2*prevalence_c302/(PR_2*prevalence_c302 - prevalence_c302 + 1),
 p_D3: PR_3*p_3*prevalence_c302/(PR_3*prevalence_c302 - prevalence_c302 + 1),
 p_S4: (PR_1*PR_2*PR_3*p_1*prevalence_c302**3 - PR_1*PR_2*PR_3*p_1*prevalence_c302**2 + PR_1*PR_2*PR_3*p_2*prevalence_c302**3 - PR_1*PR_2*PR_3*p_2*prevalence_c302**2 + PR_1*PR_2*PR_3*p_3*prevalence_c302**3 - PR_1*PR_2*PR_3*p_3*prevalence_c302**2 - PR_1*PR_2*PR_3*prevalence_c302**4 + PR_1*PR_2*PR_3*prevalence_c302**3 - PR_1*PR_2*p_1*prevalence_c302**3 + 2*PR_1*PR_2*p_1*prevalence_c302**2 - PR_1*PR_2*p_1*prevalence_c302 - PR_1*PR_2*p_2*prevalence_c302**3 + 2*PR_1*PR_2*p_2*prevalence_c302**2 - PR_1*PR_2*p_2*prevalence

In [18]:
PR_1=1.041978
PR_2=1.005639
PR_3=1.006994
PR_4=0.998314

prevalence_c302 = 0.03
p_1 = 0.016
p_2 = 0.17
p_3 = 0.4
p_4 = 1-p_1-p_2-p_3

In [19]:
p_D1 = (PR_1 * p_1 * prevalence_c302) / (PR_1 * prevalence_c302 - prevalence_c302 + 1)
p_D2 = (PR_2 * p_2 * prevalence_c302) / (PR_2 * prevalence_c302 - prevalence_c302 + 1)
p_D3 = (PR_3 * p_3 * prevalence_c302) / (PR_3 * prevalence_c302 - prevalence_c302 + 1)
p_S1 = (-p_1 * prevalence_c302 + p_1) / (PR_1 * prevalence_c302 - prevalence_c302 + 1)
p_S2 = (-p_2 * prevalence_c302 + p_2) / (PR_2 * prevalence_c302 - prevalence_c302 + 1)
p_S3 = (-p_3 * prevalence_c302 + p_3) / (PR_3 * prevalence_c302 - prevalence_c302 + 1)
p_D4 = prevalence_c302 - p_D1 - p_D2 - p_D3
p_S4 = (1 - prevalence_c302) - p_S1 - p_S2 - p_S3

In [20]:
import math

In [21]:
assert math.isclose(p_D1+p_D2+p_D3+p_D4+p_S1+p_S2+p_S3+p_S4, 1, abs_tol=1e-8), 'Do not all sum to one'
for i in [p_D1,p_D2,p_D3,p_D4,p_S1,p_S2,p_S3,p_S4]:
    assert i > 0, f'{i} is negative'
    assert i < 1, f'{i} is greater than one'

In [22]:
print(p_D1,
     p_D2,
     p_D3,
     p_D4)

0.0004995203740121915 0.00512789141460939 0.012081393082103513 0.012291195129274903


In [23]:
print(p_S1,p_S2,p_S3,p_S4)

0.015500479625987808 0.16487210858539064 0.3879186069178965 0.401708804870725


In [24]:
p_D4 / (p_D1 + p_D2 + p_D3 + p_D4)

0.40970650430916344

In [25]:
p_S4 / (p_S1 + p_S2 + p_S3 + p_S4)

0.41413278852652063